In [1]:
!pip install accelerate -U

In [2]:
pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch transformers safetensors

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from safetensors.torch import load_file as load_safetensors

In [5]:
!git clone https://github.com/ryanzhumich/AESLC

Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461
Receiving objects: 100% (17469/17469), 7.36 MiB | 19.73 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [12]:
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import Trainer, TrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader


2024-07-20 14:52:35.131626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-20 14:52:35.131761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-20 14:52:35.264770: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# Function for text preprocessing
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())

  
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalnum()]

    
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text



In [9]:
class EmailSubjectDataset(Dataset):
    def __init__(self, emails, subjects, tokenizer, max_length):
        self.emails = emails
        self.subjects = subjects
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.emails)

    def __getitem__(self, idx):
        email_text = self.emails[idx]
        subject_text = self.subjects[idx]

        # Tokenize inputs and outputs
        inputs = self.tokenizer(email_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                padding='max_length')
        outputs = self.tokenizer(subject_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                 padding='max_length')

        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten(),
            'labels': outputs.input_ids.flatten()
        }
    
    


In [10]:



dir = '/kaggle/working/AESLC/enron_subject_line/train'


model_name = 't5-small' 
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Load emails and subjects from files
emails = []
subjects = []

for filename in os.listdir(dir):

    with open(os.path.join(dir, filename), 'r', encoding='utf-8', errors='ignore') as file:
        email_text = file.read()
        # Extract subject from email body using regex
        match = re.search(r'@subject\s+(.*)', email_text, re.IGNORECASE | re.DOTALL)

        if match:
            subject = match.group(1).strip()
            email_body = re.sub(r'@subject\n(.+)\n', '', email_text).strip()
            preprocessed_email = preprocess_text(email_body)

            emails.append(preprocessed_email)
            subjects.append(subject)

# Create dataset
print(emails[0])
print(subjects[0])
dataset = EmailSubjectDataset(emails, subjects, tokenizer, max_length=512)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

beginning updated process reporting notification violation executive report viewer reporting process eliminates need m word m excel document place word excel document receive link report viewer access executive report viewer question access level please contact chris abel thank subject process change violation notification reporting
Process Change To Violations and Notifications Reporting


In [13]:



# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy="epoch",
)


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Fine-tuning the model
trainer.train()



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.043600,0.032823
2,0.034800,0.029789
3,0.032100,0.029069


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4332, training_loss=0.1038988268386404, metrics={'train_runtime': 2209.0146, 'train_samples_per_second': 15.683, 'train_steps_per_second': 1.961, 'total_flos': 4688781370195968.0, 'train_loss': 0.1038988268386404, 'epoch': 3.0})

In [ ]:
model_path = "/kaggle/input/t5-small/tensorflow2/default/1"
state_dict = load_safetensors(f"{model_path}/model.safetensors")
model = T5ForConditionalGeneration.from_pretrained(model_path, state_dict=state_dict)
tokenizer = T5Tokenizer.from_pretrained(model_path)


In [26]:
from torch.utils.data import Subset


subset_size = 200


val_subset_indices = list(range(min(subset_size, len(val_dataset))))
val_subset = Subset(val_dataset, val_subset_indices)


val_dataloader = DataLoader(val_subset, batch_size=4, shuffle=False)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model.to(device)
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

     
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=1024,
            num_beams=5,
            early_stopping=True
        )

       
        generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        reference_texts = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]

        predictions.extend(generated_texts)
        references.extend(reference_texts)

In [28]:
!pip install rouge_score
!pip install meteor
!pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=78377dfc378c831a7d076916fcceaaf8c34c603bc59ad7be804390bbb585f8af
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 79.3 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.4 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.7/776.7

In [39]:
import evaluate
rouge = evaluate.load('rouge')


In [38]:
result = rouge.compute(predictions=predictions, references=references)
results_sacrebleu = sacrebleu.compute(predictions=predictions, references=references, lowercase = True)
#results_meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)
# print(result)
# print(results_sacrebleu)
print("R1:", round(result["rouge1"], 4))
print("R2:", round(result["rouge2"], 4))
print("RL:", round(result["rougeL"], 4))
print("RLsum:", round(result["rougeLsum"], 4))



R1: 0.7512
R2: 0.4609
RL: 0.7526
RLsum: 0.7518


In [18]:

local_model_dir = './saved_model'
trainer.save_model(local_model_dir)

In [21]:
!zip -r /kaggle/working/saved_model.zip /kaggle/working/saved_model

  adding: kaggle/working/saved_model/ (stored 0%)
  adding: kaggle/working/saved_model/training_args.bin (deflated 52%)
  adding: kaggle/working/saved_model/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/saved_model/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/saved_model/model.safetensors (deflated 10%)
  adding: kaggle/working/saved_model/generation_config.json (deflated 29%)
  adding: kaggle/working/saved_model/added_tokens.json (deflated 83%)
  adding: kaggle/working/saved_model/spiece.model (deflated 48%)
  adding: kaggle/working/saved_model/config.json (deflated 62%)


In [23]:
def subject_gen_func(model, tokenizer, email, max_length=512):
    device = next(model.parameters()).device

    email = preprocess_text(email)

    input_ids = tokenizer.encode(email, return_tensors='pt').to(device)  # Move input_ids to device

    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id
    output_ids = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True ,num_return_sequences=1,attention_mask=attention_mask,
            pad_token_id=pad_token_id)


    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

def words_split(text):
    words = text.split()
    
    return ' '.join(words[:5])

In [24]:
mail = "Greg/Phillip,  Attached is the Grande Communications Service Agreement. The business points can be found in Exhibit C.  I Can get the Non-Disturbance agreement after it has been executed by you and Grande. I will fill in the Legal description of the property one I have received it. Please execute and send to:  Grande Communications, 401 Carlson Circle, San Marcos Texas, 78666 Attention Hunter Williams."
subj = subject_gen_func(model, tokenizer, mail)
# print(subj)
print("Generated Subject:", words_split(subj))



mail=' I will have my cell with me until Saturday the 22nd. 713-569-4684. From the 22-29th please reach me at 781-830-9199. I will be out of the country but checking in at that number. I will call you tonight to make sure that this information has made it to you. Thanks-Jeb '
subj = subject_gen_func(model, tokenizer, mail)
print("Generated Subject:", words_split(subj))




mail=' I left a message earlier with your assistant. My contact information is below, please touch base with me as soon as possible. I am working on a project that is very pressing.'
subj = subject_gen_func(model, tokenizer, mail)
print("Generated Subject:", words_split(subj))



mail = "Liane,   As we discussed yesterday, I am concerned there has been an attempt to  manipulate the  El Paso San Juan monthly index. A single buyer entered the  marketplace on both September 26 and 27 and paid above market prices  ($4.70-$4.80) for San Juan gas with the intent to distort the index. At the  time of these trades, offers for physical gas at significantly (10 to 15  cents) lower prices were bypassed in order to establish higher trades to  report into the index calculation. Additionally, these trades are out of  line with the associated financial swaps for San Juan. We have compiled a list of financial and physical trades executed from  September 25 to September 27. These are the complete list of trades from  Enron Online (EOL), Enron's direct phone conversations, and three brokerage  firms (Amerex, APB, and Prebon). Please see the attached spreadsheet for a  trade by trade list and a summary. We have also included a summary of gas  daily prices to illustrate the value of San Juan based on several spread  relationships. The two key points from this data are as follows:   1. The high physical prices on the 26th & 27th (4.75,4,80) are much greater  than the high financial trades (4.6375,4.665) on those days. 2. The spread relationship between San Juan and other points (Socal &  Northwest)  is  consistent between the end of September and         October gas daily. It doesn't make sense to have monthly indeces that  are dramatically different. I understand you review the trades submitted for outliers. Hopefully, the  trades submitted will reveal counterparty names and you will be able to  determine that there was only one buyer in the 4.70's and these trades are  outliers. I wanted to give you some additional points of reference to aid in  establishing a reasonable index. It is Enron's belief that the trades at  $4.70 and higher  were above market trades that should be excluded from the  calculation of index. It is our desire to have reliable and accurate indeces against which to  conduct our physical and financial business. Please contact me  anytime I can assist you towards this goal. Sincerely, "
subj = subject_gen_func(model, tokenizer, mail)
print("Generated Subject:", words_split(subj))




Generated Subject: Great Communication 401 Carlson Circle,
Generated Subject: Saturday 22nd, please reach country
Generated Subject: Work Project Pressing
Generated Subject: I'm sincerely pursuing my goals


In [29]:
generated_texts

['sara draft certificate sight draft niagara mohawk el paso electric lg e energy marketing energy production case renewed put together file include copy lc underlying trading doc correspondence sent except niagara mohawk requires copy invoice certificate letterhead appear special requirement leslie reef faxing invoice u tomorrow around thanks help doc file called enron restructuring subject lc doc',
 'tricia want exercise enron stock option know call name phone number someone also know information need order exercise could give call x6910 help thanx subject exercising stock option',
 'please format text file processed grms following manner 1 sort book id master deal id reference date 2 date must following format 3 book name must every day 4 cell contain number please put zero value grms recognize field supposed contain number 5 please name file manner consistent question feel free give call thanks subject power exotics txt file',
 'spoke miriam radich tiger lawyer advised ect name chan